In [1]:
import pandas as pd
import geopy.distance
import json
import datetime

### Bike

In [2]:
data = pd.read_csv('data/unprocessed/bikes_2018_q1.csv')

In [3]:
data.head()

,Unnamed: 0,duration_sec,start_time,end_time,start_station_id,start_station_latitude,start_station_longitude,end_station_id,end_station_latitude,end_station_longitude
0,0,75284,2018-01-31 22:52:35.2390,2018-02-01 19:47:19.8240,120,37.761420,-122.426435,285,37.783521,-122.431158
1,1,85422,2018-01-31 16:13:34.3510,2018-02-01 15:57:17.3100,15,37.795392,-122.394203,15,37.795392,-122.394203
2,2,71576,2018-01-31 14:23:55.8890,2018-02-01 10:16:52.1160,304,37.348759,-121.894798,296,37.325998,-121.877120
3,3,61076,2018-01-31 14:53:23.5620,2018-02-01 07:51:20.5000,75,37.773793,-122.421239,47,37.780955,-122.399749
4,4,39966,2018-01-31 19:52:24.6670,2018-02-01 06:58:31.0530,74,37.776435,-122.426244,19,37.788975,-122.403452


In [4]:
data.shape

(312902, 10)

Filtering out return trips - this is redundant, will be done more precisely later

In [5]:
data = data[data['start_station_id'] != data['end_station_id']]

In [6]:
data.shape

(305111, 10)

Filtering out trips starting / ending outside SF

In [7]:
station_data = pd.read_csv('data/sf_stations.csv')

In [8]:
station_ids = station_data.id.unique()

In [9]:
data = data[data['start_station_id'].isin(station_ids) & data['end_station_id'].isin(station_ids)]

In [10]:
data.shape

(228569, 10)

Remove very slow (possibly having breaks) and return trips

In [11]:
start_lat = 'start_station_latitude'
start_lon = 'start_station_longitude'
end_lat = 'end_station_latitude'
end_lon = 'end_station_longitude'

def get_row_distance(row):
    return geopy.distance.distance((row[start_lat], row[start_lon]), (row[end_lat], row[end_lon])).km

In [12]:
data['distance'] = data.apply(get_row_distance, axis=1)

In [13]:
data['speed'] = data['distance'] / data['duration_sec'] * 3600

In [14]:
data = data[data['distance'] > 0.3]

In [15]:
data = data[data['speed'] > 1]

In [16]:
data.shape

(224745, 12)

Add zone ids

In [17]:
with open('data/station2zone.json', 'r') as f:
    zones = json.load(f)

In [18]:
data['start_zone'] = data.apply(lambda x: zones[str(x.start_station_id)], axis=1)

In [19]:
data['end_zone'] = data.apply(lambda x: zones[str(x.end_station_id)], axis=1)

Proper dates

In [20]:
data['start_date'] = data.apply(lambda x: datetime.datetime.strptime(x.start_time, '%Y-%m-%d %H:%M:%S.%f'), axis=1)

In [21]:
data['hour_of_day'] = data.start_date.dt.hour

In [22]:
data['day_of_week'] = data.start_date.dt.dayofweek

In [23]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [24]:
data['day_of_week'] = data.apply(lambda x: days[x.day_of_week], axis=1)

Extract csvs

In [25]:
cols = list(data.columns)
cols[1] = 'duration'
cols[4] = 'start_station'
cols[5] = 'start_station_lat'
cols[6] = 'start_station_lon'
cols[7] = 'end_station'
data.columns = cols

In [26]:
station_cols = ['start_station', 'end_station', 'duration']
station_dow_cols = ['day_of_week'] + station_cols
station_hod_cols = ['hour_of_day'] + station_cols
zone_cols = ['start_zone', 'end_zone', 'duration']
zone_dow_cols = ['day_of_week'] + zone_cols
zone_hod_cols = ['hour_of_day'] + zone_cols
common_cols = ['start_zone', 'end_zone', 'start_station', 'end_station', 'duration']

In [27]:
bike_stations_hod = data[station_hod_cols].groupby(station_hod_cols[:-1]).mean()

In [28]:
bike_stations_hod.head()

duration
hour_of_day start_station end_station          
0           3             11              499.0
                          19              114.0
                          22              435.0
                          23             1412.0
                          30              356.0

In [29]:
bike_stations_hod.to_csv('data/bike_stations_hod.csv')

In [30]:
bike_zones_hod = data[zone_hod_cols].groupby(zone_hod_cols[:-1]).mean()

In [31]:
bike_zones_hod.head(5)

duration
hour_of_day start_zone end_zone          
0           80         80           225.5
                       113          555.0
                       212         1235.0
                       1246         605.0
            113        537          312.0

In [32]:
bike_zones_hod.to_csv('data/bike_zones_hod.csv')

In [33]:
bike_stations_dow = data[station_dow_cols].groupby(station_dow_cols[:-1]).mean()

In [34]:
bike_stations_dow.head(5)

duration
day_of_week start_station end_station          
Friday      3             4               351.0
                          5               278.5
                          6              1643.2
                          8              1300.6
                          9               695.0

In [35]:
bike_stations_dow.to_csv('data/bike_stations_dow.csv')

In [36]:
bike_zones_dow = data[zone_dow_cols].groupby(zone_dow_cols[:-1]).mean()

In [37]:
bike_zones_dow.head(5)

duration
day_of_week start_zone end_zone             
Friday      80         80         293.555556
                       113        462.937500
                       206        762.904762
                       207       1795.333333
                       208       1364.333333

In [38]:
bike_zones_dow.to_csv('data/bike_zones_dow.csv')

### Uber

In [39]:
uber_dow = pd.read_csv('data/unprocessed/uber_2018_q1_day_of_week.csv')

In [40]:
uber_dow.head()

,Unnamed: 0,sourceid,dstid,dow,mean_travel_time
0,0,1829,2594,4,1615.17
1,1,1919,1694,4,3981.88
2,2,2356,331,3,965.52
3,3,2308,811,3,1603.71
4,4,2323,661,3,334.06


In [41]:
uber_dow.shape

(3809802, 5)

In [42]:
with open('data/sf_zones.json', 'r') as f:
    zone_data = json.load(f)

In [43]:
target_zones = [zone['properties']['id'] for zone in zone_data['features']]

In [44]:
uber_dow = uber_dow[uber_dow['sourceid'].isin(target_zones) & uber_dow['dstid'].isin(target_zones)]

In [45]:
uber_dow.shape

(259529, 5)

In [46]:
uber_dow['day_of_week'] = uber_dow.apply(lambda x: days[int(x.dow) - 1], axis=1)

In [47]:
uber_dow = uber_dow[['sourceid', 'dstid', 'mean_travel_time', 'day_of_week']]

In [48]:
uber_dow.columns = ['start_zone', 'end_zone', 'duration', 'day_of_week']
uber_dow.duration = uber_dow.duration.astype('int')

In [49]:
uber_dow.head()

,start_zone,end_zone,duration,day_of_week
4,2323,661,334,Wednesday
17,1293,1772,277,Saturday
21,1298,1722,106,Saturday
22,2466,649,785,Monday
29,867,534,336,Friday


In [50]:
uber_dow.to_csv('data/uber_dow.csv', index=None)

And now hour of the day

In [51]:
uber_hod = pd.read_csv('data/unprocessed/uber_2018_q1_hour_of_day.csv')

In [52]:
uber_hod = uber_hod[uber_hod['sourceid'].isin(target_zones) & uber_hod['dstid'].isin(target_zones)]

In [53]:
uber_hod = uber_hod[['sourceid', 'dstid', 'mean_travel_time', 'hod']]

In [54]:
uber_hod.columns = ['start_zone', 'end_zone', 'duration', 'hour_of_day']
uber_hod.duration = uber_hod.duration.astype('int')
uber_hod.head()

,start_zone,end_zone,duration,hour_of_day
10,9,21,641,4
16,9,44,896,5
25,9,78,1156,22
26,9,82,557,3
31,21,9,753,4


In [55]:
uber_hod.to_csv('data/uber_hod.csv', index=None)